### PROJETO 1 REDES SOCIAIS

**GRUPO D:** 
- ESTHER DAGIR
- FELIPE LEMOS
- JV PAZOTTI

**DESCRIÇÃO DATASET CORD19**

Artigos acadêmicos relacionados ao COVID-19 e suas variantes que integram o dataset CORD-19. Coleção mais extensa (+134.000 artigos) de pesquisa sobre coronavírus legível por máquina disponível para mineração de dados até o momento, solicitada pelo governo americano em conjunto com entidades de pesquisa sobre o vírus e suas variantes. 

### Importando as Bibliotecas


In [ ]:
import pandas as pd
import seaborn as sns
import netpixi
from netpixi.integration.gt import *
from regression.integration.gt import *
from collections import defaultdict
from itertools import combinations

### Dataset

In [ ]:
data = pd.read_csv("metadata.csv").sample(frac=1/100, random_state=42)

In [ ]:
data.info()

In [ ]:
data=data.copy().drop(columns=['pdf_json_files','pmc_json_files','arxiv_id','pmcid','mag_id','s2_id'])

In [ ]:
data['authors'] = data['authors'].astype(str)

### Construindo a rede de coautoria

In [ ]:
g = Graph(directed=False) # não dirigido

In [ ]:
collabs = {}

# percorre todos os autores no dataset
for authors in data['authors']:
    # divide os autores em uma lista
    authors_list = authors.split('; ')
    # percorre todos os pares de autores na lista
    for i in range(len(authors_list)):
        for j in range(i+1, len(authors_list)):
            # ordena os nomes dos autores em ordem alfabética para evitar duplicatas
            pair = tuple(sorted([authors_list[i], authors_list[j]]))
            # incrementa o contador de co-autorias para esse par de autores
            collabs[pair] = collabs.get(pair, 0) + 1    
            
co_authors = [] #todos os influentes

# seleciona os autores que fizeram pelo menos 2 co-autorias (threshold)
for pair, count in collabs.items():
    if count >= 2: 
        # adiciona os autores da co-autoria à lista de autores colaboradores influentes
        co_authors.extend(pair)
        
unique_authors = set(co_authors)

In [ ]:
for author in unique_authors:
    g.add_vertex(author) 
    
for pair, count in collabs.items():
    if count >= 2 and pair[0] in unique_authors and pair[1] in unique_authors:
        g.add_edge(pair[0], pair[1])

In [ ]:
cord19 = gt_data(g)

In [ ]:
n = g.num_vertices()
m =  g.num_edges()
max_edges = n * (n - 1) if g.is_directed() else n * (n - 1) // 2
d = m / max_edges

print('Número de vértices de g:', n)
print('Número de arestas de g:', m)
print('Máximo possível de arestas de g:', max_edges)
print('Densidade de g:', d)

In [ ]:
in_degrees = [] 
for v in g.all_vertices():
    in_degrees.append(v.in_degree())

out_degrees = []
for v in g.all_vertices():
    out_degrees.append(v.out_degree())

degrees = []
for v in g.all_vertices():
    degrees.append(v.total_degree())
    
cord19['in_degree'] = in_degrees
cord19['out_degree'] = out_degrees
cord19['degree'] = degrees
cord19

In [ ]:
cord19.to_csv('cord19.csv')

In [ ]:
sns.histplot(cord19['degree'])
cord19['degree'].describe()

In [ ]:
gt_save(g, './cord19_graph.net.gz')
g = gt_load('./cord19_graph.net.gz')
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
netpixi.render('./cord19_graph.net.gz');

In [ ]:
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, './cord19_graph.net.gz')
r = netpixi.render('./cord19_graph.net.gz', infinite=True);